In [ ]:

import paramiko
import os
import threading
import time
import json
from queue import Queue


# Log of transferred files
transferred_files_log = []

def connect_ssh(host, port, username, password):
    """Establish an SSH connection."""
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    try:
        client.connect(host, port=port, username=username, password=password)
        return client
    except Exception as e:
        print(f"Failed to connect to {host}: {e}")
        return None

def transfer_file(file_info, source_ssh, target_ssh):
    """Stream a file directly from the source to the target using SFTP channels."""
    source_file, target_file = file_info

    try:
        source_sftp = source_ssh.open_sftp()
        target_sftp = target_ssh.open_sftp()

        source_size = source_sftp.stat(source_file).st_size

        try:
            target_size = target_sftp.stat(target_file).st_size
        except FileNotFoundError:
            target_size = -1

        if target_size == source_size:
            print(f"Skipping {source_file}, already transferred.")
            return
        elif target_size > 0:
            print(f"Deleting incomplete file: {target_file}")
            target_sftp.remove(target_file)

        print(f"Transferring {source_file} → {target_file}")
        with source_sftp.file(source_file, "rb") as src_file:
            target_sftp.putfo(src_file, target_file)
        print(f"Transferred {source_file} → {target_file}")

        transferred_files_log.append(target_file)

        source_sftp.close()
        target_sftp.close()
    except Exception as e:
        print(f"Error transferring {source_file}: {e}")

def transfer_files(folder_input, target_path, source_ssh, target_ssh, num_threads=4):
    """Scan and transfer files from source to target server."""
    print(f"Checking if {folder_input} exists on the source server...")
    stdin, stdout, stderr = source_ssh.exec_command(f"ls -ld {folder_input} 2>&1")
    folder_info = stdout.read().decode().strip()
    if "No such file or directory" in folder_info:
        print(f"Error: {folder_info}")
        return
    print(f"Folder exists: {folder_info}")

    print(f"Scanning files in {folder_input}...")
    stdin, stdout, stderr = source_ssh.exec_command(f"find {folder_input} -type f 2>/dev/null")
    file_list = stdout.read().decode().splitlines()
    if not file_list:
        print(f"No files found in {folder_input}.")
        return

    print(f"Found {len(file_list)} files to transfer.")

    file_queue = Queue()

    for file in file_list:
        relative_path = os.path.relpath(file, folder_input)
        target_file = f"{target_path}/{relative_path}"
        file_queue.put((file, target_file))

    def worker():
        while not file_queue.empty():
            file_info = file_queue.get()
            transfer_file(file_info, source_ssh, target_ssh)
            file_queue.task_done()

    threads = []
    for _ in range(num_threads):
        thread = threading.Thread(target=worker)
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()

def save_transferred_files_log(log_file="transferred_files_log.json"):
    """Save the log of transferred files."""
    with open(log_file, "w") as f:
        json.dump(transferred_files_log, f, indent=4)

def main():
    """Main execution function."""
    source_server = {
        'host': 'SOURCE_HOST',
        'port': 22,
        'username': 'SOURCE_USER',
        'password': 'SOURCE_PASSWORD'
    }
    target_server = {
        'host': 'TARGET_HOST',
        'port': 22,
        'username': 'TARGET_USER',
        'password': 'TARGET_PASSWORD'
    }

    target_path = '/path/to/target/directory'
    folder_input = '/path/to/source/directory'

    start_time = time.time()

    source_ssh = connect_ssh(**source_server)
    target_ssh = connect_ssh(**target_server)

    if source_ssh is None or target_ssh is None:
        print("Failed to establish SSH connections. Exiting.")
        return

    print(f"Starting file transfer from {folder_input} to {target_path}...")
    transfer_files(folder_input, target_path, source_ssh, target_ssh, num_threads=4)

    save_transferred_files_log()

    source_ssh.close()
    target_ssh.close()

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Completed file transfer in {elapsed_time:.2f} seconds")

if __name__ == "__main__":
    main()


In [ ]:
# File Validation Script: Remote SSH File Comparison

In [ ]:
import paramiko
import os
from openpyxl import Workbook
from openpyxl.styles import PatternFill
import threading


# SSH Connection

def connect_ssh(host, port, username, password):
    """Establish an SSH connection."""
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    try:
        client.connect(host, port=port, username=username, password=password)
        return client
    except Exception as e:
        print(f"Failed to connect to {host}: {e}")
        return None

# List files on remote server

def list_files(ssh_client, path, result):
    """Retrieve a list of files from a remote directory."""
    try:
        stdin, stdout, stderr = ssh_client.exec_command(f'find {path} -type f 2>/dev/null')
        files = stdout.read().decode().splitlines()
        result.extend([os.path.basename(file) for file in files])
    except Exception as e:
        print(f"Error listing files in {path}: {e}")

# Validate files

def validate_files(source_files, target_files):
    """Compare files between source and target directories."""
    file_counts = {}
    for file in target_files:
        file_counts[file] = file_counts.get(file, 0) + 1

    missing_files = set(source_files) - set(target_files)
    extra_files = set(target_files) - set(source_files)

    return file_counts, missing_files, extra_files

# Save validation report

def save_validation_report(file_counts, missing_files, extra_files, report_file="file_validation_report.xlsx"):
    """Generate an Excel report for file validation results."""
    wb = Workbook()
    ws = wb.active
    ws.append(["File Name", "Status"])

    yellow_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")
    red_fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")

    for file, count in file_counts.items():
        if count == 1:
            ws.append([file, "Single file - Size match"])
            ws[f"B{ws.max_row}"].fill = yellow_fill
        elif count > 1:
            ws.append([file, f"Duplicate file - Found {count} times"])
            ws[f"B{ws.max_row}"].fill = red_fill

    for file in missing_files:
        ws.append([file, "Missing in target"])
        ws[f"B{ws.max_row}"].fill = red_fill

    for file in extra_files:
        ws.append([file, "Extra file in target"])

    wb.save(report_file)
    print(f"Validation report saved to {report_file}")

# Main function

def main():
    """Main execution function."""
    source_server = {
        'host': 'SOURCE_HOST',
        'port': 22,
        'username': 'SOURCE_USER',
        'password': 'SOURCE_PASSWORD'
    }
    target_server = {
        'host': 'TARGET_HOST',
        'port': 22,
        'username': 'TARGET_USER',
        'password': 'TARGET_PASSWORD'
    }

    source_path = '/path/to/source/directory'
    target_path = '/path/to/target/directory'

    source_ssh = connect_ssh(**source_server)
    target_ssh = connect_ssh(**target_server)

    if source_ssh is None or target_ssh is None:
        print("Failed to establish SSH connections. Exiting.")
        return

    source_files = []
    target_files = []

    # Parallel file listing
    source_thread = threading.Thread(target=list_files, args=(source_ssh, source_path, source_files))
    target_thread = threading.Thread(target=list_files, args=(target_ssh, target_path, target_files))

    source_thread.start()
    target_thread.start()

    source_thread.join()
    target_thread.join()

    file_counts, missing_files, extra_files = validate_files(source_files, target_files)
    save_validation_report(file_counts, missing_files, extra_files)

    source_ssh.close()
    target_ssh.close()

if __name__ == "__main__":
    main()
